In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm.auto import tqdm

c:\Users\ryanp\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
N_NODES = 5
LOWER_BOUND = (-N_NODES//2)
UPPER_BOUND = (N_NODES//2)+1
print(f"Nodes: {N_NODES}\nLower bound: {LOWER_BOUND}\nUpper bound: {UPPER_BOUND}")
alphas = [.15, .1, .05]
GAMMA = 0.9

Nodes: 5
Lower bound: -3
Upper bound: 3


In [3]:
class Env():
    def __init__(self) -> None:
        # transition reward for each action pair
        # for each state, contains [reward_left, reward_right]
        rewards = np.zeros((N_NODES, 2), dtype=int)
        # only moving right from the last state gives a reward
        rewards[-1,1] = 1
        self.rewards = rewards
        self.states = [i for i in range(LOWER_BOUND, UPPER_BOUND+1)]
        # nodes = {state:transition_rewards[left, right]}
        self.nodes = {
            state:reward for state,reward in zip(self.states[1:],rewards)
        }

In [4]:
class Agent():
    def __init__(self) -> None:
        self.position = 0
        self.done = False
        self.action = None
        self.env = Env()
        self.values = {state:value for state, value in \
                                        zip(self.env.states, np.zeros(len(self.env.states)))}
        
    def step(self):
        """
        Performs a step according to the uniform random policy
        """
        s = self.position
        self.action = np.random.choice([0,1])
        if self.action == 1:
            self.position += 1
        else: self.position -= 1
        
        # check terminal state
        if self.position in [LOWER_BOUND, UPPER_BOUND]:
            self.done = True

        # s, s_prime, action
        return s, self.position, self.action
    
    def reset(self):
        self.position = 0
        self.done = False
        return self.position
    
    def rmse(self):
        """
        Returns the root mean squarred error 
        The error is considered as the difference between the prediction
        and the true state value
        """
        true_value_function = np.arange(1,N_NODES+1)/(N_NODES+1)
        delta = list(self.values.values())[1:-1] - true_value_function
        rmse = np.sqrt(np.mean(np.power(delta, 2)))
        return np.round(rmse,4)

In [5]:
class TD_Agent(Agent):
    def __init__(self, alpha:float, gamma:float) -> None:
        self.alpha = alpha
        self.gamma = gamma
        super().__init__()

    def update_value(self, s, s_prime, reward):
        """
        Update the state values according to the TD(0) algorithm
        The terminal state has no value, therefore the update is modified for s_prime = terminal
        """
        if not self.done:
            self.values[s] = self.values[s] + self.alpha*(reward + self.gamma * self.values[s_prime] - self.values[s])
        else: 
            self.values[s] = self.values[s] + self.alpha*(reward - self.values[s])

    def play_episode(self):
        s = self.reset() # initial state
        while True:
            s, s_prime, action = self.step()
            reward = self.env.nodes.get(s)[action]
            self.update_value(s, s_prime, reward)
            if self.done:
                break
            s = s_prime

In [6]:
def get_runs(alphas, n_runs=10, n_episodes=100):
    """
    For each value of alpha, performs n_runs of n_episodes and records the error
    Runs for a specific alpha value are exported in a dataframe
    """
    errors = {}
    dataframes = {}
    for alpha in alphas:
        print(f'Computing errors for alpha: {alpha}')
        for run in tqdm(range(n_runs)):
            errors[f'{alpha}_{run}'] = []
            dataframes[f'{alpha}'] = pd.DataFrame()
            a = TD_Agent(alpha=alpha, gamma=GAMMA)
            for _ in range(n_episodes):
                a.play_episode()
                errors[f'{alpha}_{run}'].append(a.rmse())
            dataframes[f'{alpha}'] = pd.concat((dataframes[f'{alpha}'], pd.Series(errors)), axis=1)
    return dataframes


In [7]:
def average_run(dataframe:pd.DataFrame):
    """
    Return the average error per episode for each alpha
    """
    return pd.DataFrame(dataframe.to_list(), columns=range(100)).apply(np.mean)


In [8]:
def plot_runs(dataframes):
    """
    Plots the average error per episode for each alpha
    """
    averaged_runs = pd.DataFrame([average_run(dataframes[str(alpha)][0]) for alpha in alphas]).T
    averaged_runs.columns = alphas
    fig = px.line(averaged_runs,
            labels={'index':'Number of episodes', 'value':'RMSE', 'variable':'Alpha'},
            title="Root mean square errors of value estimations for 10 runs of TD(0)",
            height=600
            )
    fig.update_layout()
    fig.show()


In [9]:
errors = get_runs(alphas, n_runs=1000, n_episodes=100)
plot_runs(errors)

Computing errors for alpha: 0.15


100%|██████████| 1000/1000 [00:13<00:00, 73.94it/s]


Computing errors for alpha: 0.1


100%|██████████| 1000/1000 [00:13<00:00, 72.54it/s]


Computing errors for alpha: 0.05


100%|██████████| 1000/1000 [00:13<00:00, 72.18it/s]


In [165]:
class Episode():
    def __init__(self, initial_state=0) -> None:
        self.initial_state = initial_state
        self.record = {
            'states' :  [self.initial_state],
            'actions' : [],
            'rewards' : [],
        }

    def add_action_reward(self, action, reward):
        self.record['actions'].append(action)
        self.record['rewards'].append(reward)

    def add_state(self, state):
        self.record['states'].append(state)

    def reverse(self):
        """
        Reverses the states, actions and rewards lists in an episode
        MC updates are made backwards, from the last step to the first
        """
        return {key: list(reversed(values)) for (key, values) in self.record.items()}

    def episode_generator(self):
        """
        Generator yielding (state, action, rewards) starting from 
        the end of the episode
        """
        rev = self.reverse()
        print(f'N steps: {self.n_steps}')
        iterator = (list(
                        zip(
                            rev['states'], 
                            rev['actions'], 
                            rev['rewards'])
                        )
                    )
        for idx in range(self.n_steps-1):
            yield iterator[idx]

class MC_Agent(Agent):
    def __init__(self, alpha:float, gamma:float) -> None:
        self.alpha = alpha
        self.gamma = gamma
        self.returns = []
        super().__init__()

    def update_value(self, episode: Episode, returns):
        """
        Update the values according to first-visit Monte Carlo,
        this approach is more suited for a deterministic environment
        """
        visited_states = []
        generator = episode.episode_generator()
        for step in range(episode.n_steps-1):
            state, action, reward = next(generator)
            if state not in visited_states:
                visited_states.append(state)
                print(f'state: {state}, action: {action} reward: {reward}')
                #returns[state] = GAMMA*returns[state] + reward
    
    def play_episode(self):
        """
        Plays one episode until termination, caches the states, actions and rewards
        At the end of the episode, updates the value of each visited state
        """
        s = self.reset() # initial state
        G = 0 # initial estimated return
        episode = Episode()

        while True:
            s, s_prime, action = self.step()
            reward = self.env.nodes.get(s)[action]
            episode.add_action_reward(action, reward)
            if self.done:
                print(f"Terminal state {s_prime}")
                break
            episode.add_state(s_prime)
            episode.n_steps = len(episode.record['states'])

        self.update_value(episode)

In [166]:
mc_a = MC_Agent(alpha=.15, gamma=.9)
mc_a.play_episode()

Terminal state 3
N steps: 5
state: 2, action: 1 reward: 1
state: 1, action: 1 reward: 0
state: 0, action: 1 reward: 0
state: 1, action: 0 reward: 0
